In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [3]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [4]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [5]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

C:\Users\WinPC\AppData\Local\Temp\ipykernel_18372\852816848.py:16: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\WinPC\AppData\Local\Temp\ipykernel_18372\852816848.py:16: FutureWarning: Pass y=[0.0000000e+00 3.0517578e-05 3.0517578e-05 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\WinPC\AppData\Local\Temp\ipykernel_18372\852816848.py:16: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00 -3.0517578e-05 ... -3.0517578e-05
 -3.0517578e-05 -3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users

In [6]:
x_train

array([[-5.22061890e+02,  3.50668907e+01,  3.75342965e+00, ...,
         1.65243153e-04,  1.04321596e-04,  6.55571566e-05],
       [-6.41227722e+02,  4.49487762e+01, -1.85174108e+00, ...,
         3.89261913e-05,  3.05255380e-05,  2.94166657e-05],
       [-6.50705750e+02,  5.30211639e+01, -4.92040443e+00, ...,
         4.75216802e-05,  3.46632514e-05,  1.62844426e-05],
       ...,
       [-5.50096191e+02,  1.70297680e+01, -1.14575644e+01, ...,
         1.51764631e-04,  1.16828531e-04,  8.47479314e-05],
       [-5.55357605e+02,  4.71569710e+01,  1.10750732e+01, ...,
         1.61086471e-04,  1.04962463e-04,  6.52811723e-05],
       [-5.04816345e+02,  3.53618660e+01, -1.43495789e+01, ...,
         6.08151546e-04,  5.55269769e-04,  4.47782222e-04]])

In [7]:
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [8]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [9]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [10]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [11]:
y_pred=model.predict(x_test)

In [12]:
y_pred

array(['happy', 'calm', 'happy', 'happy', 'fearful', 'calm', 'calm',
       'disgust', 'calm', 'happy', 'happy', 'calm', 'fearful', 'happy',
       'disgust', 'happy', 'calm', 'happy', 'happy', 'calm', 'calm',
       'disgust', 'disgust', 'calm', 'happy', 'happy', 'calm', 'happy',
       'calm', 'calm', 'happy', 'happy', 'happy', 'calm', 'happy', 'calm',
       'calm', 'fearful', 'calm', 'calm', 'happy', 'calm', 'calm', 'calm',
       'fearful', 'calm', 'calm', 'happy', 'calm', 'happy', 'fearful',
       'fearful', 'happy', 'fearful', 'happy', 'fearful', 'calm', 'happy',
       'calm', 'calm', 'disgust', 'calm', 'happy', 'calm', 'happy',
       'calm', 'calm', 'calm', 'fearful', 'happy', 'fearful', 'fearful',
       'fearful', 'fearful', 'fearful', 'disgust', 'fearful', 'happy',
       'calm', 'fearful', 'calm', 'calm', 'fearful', 'calm', 'disgust',
       'calm', 'disgust', 'fearful', 'happy', 'fearful', 'disgust',
       'calm', 'calm', 'happy', 'disgust', 'happy', 'calm', 'calm',
  

In [13]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 67.71%


In [14]:
from sklearn.metrics import accuracy_score, f1_score

In [15]:
f1_score(y_test, y_pred,average=None)

array([0.82962963, 0.40625   , 0.675     , 0.64761905])

In [16]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,happy,happy
1,calm,calm
2,happy,happy
3,happy,happy
4,disgust,fearful
5,calm,calm
6,happy,calm
7,happy,disgust
8,disgust,calm
9,happy,happy


In [17]:
import pickle
with open('model.sav', 'wb') as f:
    pickle.dump(model,f)